# 85. カーネルの可視化と解釈

**Unit 0.4: 空間の帰納バイアス - CNN & 局所性の科学**

---

## このノートブックの位置づけ

古典的フィルタを学んだ今、**CNNが実際に学習するカーネル**を覗いてみましょう。学習済みネットワークの第1層は、Gaborフィルタに似た構造を示すことが知られています。

```
Unit 0.4 学習マップ
==================

Section A: 畳み込みの世界への入口
├── 80. 畳み込みとは何か（直感編）  ✅
├── 81. 畳み込みの数学的定義  ✅
├── 82. NumPyスクラッチ実装（基礎編）  ✅
├── 83. NumPyスクラッチ実装（高速化編）  ✅
├── 84. 古典的フィルタの解剖学  ✅
└── 85. カーネルの可視化と解釈  ← 今ここ（Section A 最終）

→ 次は Section B: 受容野と階層的抽象化
```

## 学習目標

1. **学習済みCNNの第1層カーネルを可視化できる**
2. **Gaborフィルタとの類似性を理解する**
3. **深い層のカーネルが解釈困難な理由を説明できる**
4. **特徴マップの可視化を通じてCNNの「見方」を理解する**
5. **学習前後のカーネルの変化を観察できる**

## 目次

1. [学習済みCNNの中身を覗く](#1-学習済みcnnの中身を覗く)
2. [第1層のカーネル：何を見ているか](#2-第1層のカーネル何を見ているか)
3. [深い層のカーネル：解釈の困難さ](#3-深い層のカーネル解釈の困難さ)
4. [特徴マップの可視化](#4-特徴マップの可視化)
5. [学習前 vs 学習後の比較](#5-学習前-vs-学習後の比較)
6. [3DGSへの接続：学習されるガウシアン](#6-3dgsへの接続学習されるガウシアン)

---

## 環境セットアップ

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings('ignore')

# PyTorchのインポート
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torchvision import models, transforms
    from torchvision.models import VGG16_Weights, ResNet18_Weights
    TORCH_AVAILABLE = True
    print(f"PyTorch version: {torch.__version__}")
except ImportError:
    TORCH_AVAILABLE = False
    print("PyTorchが利用できません。一部の機能が制限されます。")

plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

np.random.seed(42)
print("環境準備完了")

---

## 1. 学習済みCNNの中身を覗く

ImageNetで訓練された有名なCNNモデル（VGG、ResNet）のカーネルを取り出して可視化します。

In [ ]:
if TORCH_AVAILABLE:
    # VGG16の読み込み（学習済み）
    vgg16 = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
    vgg16.eval()
    
    # ResNet18の読み込み（学習済み）
    resnet18 = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    resnet18.eval()
    
    print("モデル読み込み完了")
    print(f"\nVGG16 第1層: {vgg16.features[0]}")
    print(f"ResNet18 第1層: {resnet18.conv1}")
else:
    print("PyTorchが必要です。このセクションはスキップされます。")

In [ ]:
if TORCH_AVAILABLE:
    # VGG16の構造を確認
    print("VGG16 の畳み込み層一覧:")
    print("="*60)
    
    conv_layers = []
    for idx, layer in enumerate(vgg16.features):
        if isinstance(layer, nn.Conv2d):
            conv_layers.append((idx, layer))
            print(f"Layer {idx}: Conv2d(in={layer.in_channels}, out={layer.out_channels}, "
                  f"kernel={layer.kernel_size}, stride={layer.stride})")
    
    print(f"\n畳み込み層の総数: {len(conv_layers)}")

---

## 2. 第1層のカーネル：何を見ているか

CNNの第1層は、入力画像（RGB）から直接特徴を抽出します。このカーネルは人間にも解釈可能なパターンを示します。

In [ ]:
def visualize_first_layer_kernels(model, model_name, layer_attr):
    """
    第1層のカーネルを可視化
    
    Parameters:
    -----------
    model : nn.Module
        PyTorchモデル
    model_name : str
        モデル名（表示用）
    layer_attr : str
        第1層のアトリビュート名（例：'features[0]', 'conv1'）
    """
    # 第1層の重みを取得
    first_layer = eval(f"model.{layer_attr}")
    weights = first_layer.weight.data.cpu().numpy()
    
    # 形状: (out_channels, in_channels, H, W)
    n_filters = weights.shape[0]
    n_channels = weights.shape[1]
    
    print(f"{model_name} 第1層カーネル:")
    print(f"  形状: {weights.shape}")
    print(f"  フィルタ数: {n_filters}")
    print(f"  入力チャネル: {n_channels} (RGB)")
    print(f"  カーネルサイズ: {weights.shape[2]}×{weights.shape[3]}")
    
    # 可視化（最初の64個のフィルタ）
    n_show = min(64, n_filters)
    n_cols = 8
    n_rows = (n_show + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 2))
    axes = axes.flatten()
    
    for i in range(n_show):
        # RGBカーネルを可視化用に正規化
        kernel = weights[i]  # shape: (3, H, W)
        
        # チャネルを最後に移動 (H, W, 3)
        kernel = np.transpose(kernel, (1, 2, 0))
        
        # 0-1に正規化
        kernel = (kernel - kernel.min()) / (kernel.max() - kernel.min() + 1e-8)
        
        axes[i].imshow(kernel)
        axes[i].axis('off')
        axes[i].set_title(f'#{i}', fontsize=8)
    
    # 余ったaxesを非表示
    for i in range(n_show, len(axes)):
        axes[i].axis('off')
    
    plt.suptitle(f'{model_name} 第1層カーネル（{n_show}個を表示）', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return weights

In [ ]:
if TORCH_AVAILABLE:
    # VGG16の第1層カーネルを可視化
    vgg_kernels = visualize_first_layer_kernels(vgg16, "VGG16", "features[0]")

In [ ]:
if TORCH_AVAILABLE:
    # ResNet18の第1層カーネルを可視化
    resnet_kernels = visualize_first_layer_kernels(resnet18, "ResNet18", "conv1")

### 2.1 Gaborフィルタとの類似性

CNNの第1層カーネルは、**Gaborフィルタ**に似たパターンを示すことが知られています。Gaborフィルタは、人間の視覚野の単純細胞の応答をモデル化したものです。

In [ ]:
def gabor_kernel(size, sigma, theta, lambd, gamma, psi):
    """
    Gaborフィルタカーネルを生成
    
    Parameters:
    -----------
    size : int
        カーネルサイズ
    sigma : float
        ガウシアンエンベロープの標準偏差
    theta : float
        方向（ラジアン）
    lambd : float
        正弦波の波長
    gamma : float
        空間アスペクト比
    psi : float
        位相オフセット
    """
    center = size // 2
    x = np.arange(size) - center
    y = np.arange(size) - center
    X, Y = np.meshgrid(x, y)
    
    # 回転
    x_theta = X * np.cos(theta) + Y * np.sin(theta)
    y_theta = -X * np.sin(theta) + Y * np.cos(theta)
    
    # Gabor関数
    gaussian = np.exp(-(x_theta**2 + gamma**2 * y_theta**2) / (2 * sigma**2))
    sinusoid = np.cos(2 * np.pi * x_theta / lambd + psi)
    
    return gaussian * sinusoid

# 様々な方向のGaborフィルタを生成
fig, axes = plt.subplots(2, 8, figsize=(16, 4))

orientations = np.linspace(0, np.pi, 8, endpoint=False)

for i, theta in enumerate(orientations):
    # 実部（偶対称）
    gabor_real = gabor_kernel(11, sigma=2.0, theta=theta, lambd=4.0, gamma=0.5, psi=0)
    axes[0, i].imshow(gabor_real, cmap='RdBu')
    axes[0, i].set_title(f'{np.degrees(theta):.0f}°', fontsize=10)
    axes[0, i].axis('off')
    
    # 虚部（奇対称）
    gabor_imag = gabor_kernel(11, sigma=2.0, theta=theta, lambd=4.0, gamma=0.5, psi=np.pi/2)
    axes[1, i].imshow(gabor_imag, cmap='RdBu')
    axes[1, i].axis('off')

axes[0, 0].set_ylabel('偶対称', fontsize=10)
axes[1, 0].set_ylabel('奇対称', fontsize=10)

plt.suptitle('Gaborフィルタバンク（様々な方向）', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nGaborフィルタの特徴:")
print("- 特定の方向と周波数に選択的に応答")
print("- 人間の視覚野の単純細胞に類似")
print("- CNNの第1層は、学習によってこれに近いパターンを獲得する")

In [ ]:
if TORCH_AVAILABLE:
    # VGGカーネルとGaborの比較
    
    fig, axes = plt.subplots(2, 8, figsize=(16, 4))
    
    # 上段：VGG16の特徴的なカーネル（手動で選択）
    selected_indices = [0, 3, 7, 12, 18, 24, 32, 45]  # エッジ検出的なもの
    
    for i, idx in enumerate(selected_indices):
        kernel = vgg_kernels[idx]
        # グレースケール化（3チャネルの平均）
        kernel_gray = kernel.mean(axis=0)
        axes[0, i].imshow(kernel_gray, cmap='RdBu')
        axes[0, i].set_title(f'VGG #{idx}', fontsize=9)
        axes[0, i].axis('off')
    
    # 下段：類似のGaborフィルタ
    gabor_params = [
        (0, 4.0), (np.pi/4, 4.0), (np.pi/2, 4.0), (3*np.pi/4, 4.0),
        (0, 6.0), (np.pi/4, 6.0), (np.pi/2, 6.0), (3*np.pi/4, 6.0),
    ]
    
    for i, (theta, lambd) in enumerate(gabor_params):
        gabor = gabor_kernel(3, sigma=1.0, theta=theta, lambd=lambd, gamma=0.5, psi=0)
        axes[1, i].imshow(gabor, cmap='RdBu')
        axes[1, i].set_title(f'Gabor {np.degrees(theta):.0f}°', fontsize=9)
        axes[1, i].axis('off')
    
    axes[0, 0].set_ylabel('VGG16学習済み', fontsize=10)
    axes[1, 0].set_ylabel('Gabor（参考）', fontsize=10)
    
    plt.suptitle('学習されたカーネル vs Gaborフィルタ', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

---

## 3. 深い層のカーネル：解釈の困難さ

第2層以降のカーネルは、直接可視化しても解釈が困難です。なぜでしょうか？

In [ ]:
if TORCH_AVAILABLE:
    # 各層のカーネルサイズを確認
    print("VGG16 畳み込み層のカーネル情報:")
    print("="*70)
    print(f"{'層':>10} {'入力Ch':>10} {'出力Ch':>10} {'カーネル':>10} {'パラメータ数':>15}")
    print("-"*70)
    
    for idx, layer in conv_layers[:8]:  # 最初の8層
        w = layer.weight.data
        n_params = w.numel()
        print(f"Layer {idx:>3} {w.shape[1]:>10} {w.shape[0]:>10} "
              f"{w.shape[2]}×{w.shape[3]:>6} {n_params:>15,}")
    
    print("\n注目ポイント:")
    print("- 第1層: 3チャネル（RGB）→ 人間が解釈可能")
    print("- 第2層以降: 64チャネル以上 → 64次元空間のパターン")
    print("- 深くなるほど、カーネルは『抽象的な特徴の組み合わせ』を学習")

In [ ]:
if TORCH_AVAILABLE:
    # 第2層のカーネルを可視化（試み）
    
    layer2 = vgg16.features[2]  # 2番目の畳み込み層
    weights2 = layer2.weight.data.cpu().numpy()
    
    print(f"第2層カーネルの形状: {weights2.shape}")
    print(f"  → {weights2.shape[0]}個のフィルタ、各フィルタは{weights2.shape[1]}チャネル")
    
    # 1つのフィルタ（64チャネル）を可視化
    fig, axes = plt.subplots(8, 8, figsize=(12, 12))
    
    filter_idx = 0  # 最初のフィルタ
    kernel = weights2[filter_idx]  # shape: (64, 3, 3)
    
    for i in range(64):
        row, col = i // 8, i % 8
        channel_kernel = kernel[i]  # shape: (3, 3)
        
        # 正規化
        vmax = np.abs(channel_kernel).max()
        axes[row, col].imshow(channel_kernel, cmap='RdBu', vmin=-vmax, vmax=vmax)
        axes[row, col].axis('off')
        axes[row, col].set_title(f'Ch{i}', fontsize=7)
    
    plt.suptitle(f'第2層フィルタ#{filter_idx}の64チャネルカーネル\n（各3×3が異なる入力チャネルに対応）', 
                fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\n解釈の困難さ:")
    print("- 64個の3×3カーネルの『組み合わせ』が1つの特徴検出器")
    print("- 人間には直感的に理解できない")
    print("- だからこそ、特徴マップの可視化が重要")

---

## 4. 特徴マップの可視化

カーネルそのものではなく、**カーネルが画像に対して何に反応するか**を見ることで、CNNの動作を理解できます。

In [ ]:
def create_sample_image(size=224):
    """テスト用のサンプル画像を作成"""
    img = np.zeros((size, size, 3))
    
    # 背景（グラデーション）
    for c in range(3):
        x = np.linspace(0, 1, size)
        y = np.linspace(0, 1, size)
        X, Y = np.meshgrid(x, y)
        img[:, :, c] = 0.2 * X + 0.1 * Y + 0.1
    
    # 赤い四角形
    img[30:80, 30:80, 0] = 0.9
    img[30:80, 30:80, 1] = 0.2
    img[30:80, 30:80, 2] = 0.2
    
    # 緑の円
    yy, xx = np.ogrid[:size, :size]
    center = (150, 60)
    radius = 30
    mask = (xx - center[0])**2 + (yy - center[1])**2 <= radius**2
    img[mask, 0] = 0.2
    img[mask, 1] = 0.8
    img[mask, 2] = 0.2
    
    # 青い三角形
    for i in range(60):
        start = 130 + i // 2
        end = 190 - i // 2
        if 100 + i < size:
            img[100 + i, start:end, 0] = 0.2
            img[100 + i, start:end, 1] = 0.2
            img[100 + i, start:end, 2] = 0.9
    
    # 白黒のエッジ
    img[180:200, 30:120, :] = 0.9
    img[180:200, 120:210, :] = 0.1
    
    return np.clip(img, 0, 1)

sample_img = create_sample_image()

plt.figure(figsize=(6, 6))
plt.imshow(sample_img)
plt.title('テスト画像（様々な色・形・エッジ）', fontsize=12)
plt.axis('off')
plt.show()

In [ ]:
if TORCH_AVAILABLE:
    def get_feature_maps(model, image, layer_indices):
        """
        指定した層の特徴マップを取得
        
        Parameters:
        -----------
        model : VGG model
        image : ndarray (H, W, 3)
            0-1の範囲の画像
        layer_indices : list
            特徴マップを取得する層のインデックス
        """
        # 前処理
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])
        
        # NumPy → Tensor
        img_tensor = transform(image.astype(np.float32)).unsqueeze(0)
        
        # 各層の出力を記録
        feature_maps = {}
        x = img_tensor
        
        with torch.no_grad():
            for idx, layer in enumerate(model.features):
                x = layer(x)
                if idx in layer_indices:
                    feature_maps[idx] = x.squeeze().cpu().numpy()
        
        return feature_maps
    
    # 特徴マップを取得（第1, 2, 5, 10層）
    layer_indices = [0, 2, 5, 10]
    feature_maps = get_feature_maps(vgg16, sample_img, layer_indices)
    
    print("取得した特徴マップ:")
    for idx, fmap in feature_maps.items():
        print(f"  Layer {idx}: shape {fmap.shape}")

In [ ]:
if TORCH_AVAILABLE:
    # 特徴マップの可視化
    
    fig = plt.figure(figsize=(18, 12))
    gs = GridSpec(4, 9, figure=fig)
    
    # 元画像
    ax_orig = fig.add_subplot(gs[:2, :2])
    ax_orig.imshow(sample_img)
    ax_orig.set_title('入力画像', fontsize=12)
    ax_orig.axis('off')
    
    # 各層の特徴マップ
    layer_names = ['Layer 0\n(Conv1)', 'Layer 2\n(Conv2)', 'Layer 5\n(Conv3)', 'Layer 10\n(Conv5)']
    
    for row, (layer_idx, fmap) in enumerate(feature_maps.items()):
        # 層名
        ax_label = fig.add_subplot(gs[row, 2])
        ax_label.text(0.5, 0.5, layer_names[row], fontsize=11, ha='center', va='center')
        ax_label.axis('off')
        
        # 最初の6チャネルを表示
        for ch in range(min(6, fmap.shape[0])):
            ax = fig.add_subplot(gs[row, 3 + ch])
            ax.imshow(fmap[ch], cmap='viridis')
            ax.set_title(f'Ch{ch}', fontsize=9)
            ax.axis('off')
    
    plt.suptitle('VGG16 各層の特徴マップ（各行：1つの層、各列：異なるチャネル）', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\n観察ポイント:")
    print("- 浅い層：エッジ、色、テクスチャなど低レベル特徴に反応")
    print("- 深い層：より抽象的な特徴（物体の部品など）に反応")
    print("- 解像度が徐々に低下（プーリングの効果）")

In [ ]:
if TORCH_AVAILABLE:
    # 特定のチャネルが何に反応しているか詳しく見る
    
    fig, axes = plt.subplots(2, 5, figsize=(16, 7))
    
    layer0_fmap = feature_maps[0]  # 第1層の特徴マップ
    
    # 最も活性化が強いチャネルと弱いチャネルを見つける
    channel_activations = [layer0_fmap[ch].max() for ch in range(layer0_fmap.shape[0])]
    sorted_channels = np.argsort(channel_activations)[::-1]
    
    # 上段：活性化が強いチャネル
    axes[0, 0].imshow(sample_img)
    axes[0, 0].set_title('入力画像', fontsize=11)
    axes[0, 0].axis('off')
    
    for i, ch in enumerate(sorted_channels[:4]):
        axes[0, i+1].imshow(layer0_fmap[ch], cmap='hot')
        axes[0, i+1].set_title(f'Ch{ch} (高活性)', fontsize=11)
        axes[0, i+1].axis('off')
    
    # 下段：対応するカーネル
    axes[1, 0].axis('off')
    axes[1, 0].text(0.5, 0.5, '対応する\nカーネル', fontsize=12, ha='center', va='center')
    
    for i, ch in enumerate(sorted_channels[:4]):
        kernel = vgg_kernels[ch]
        kernel_vis = np.transpose(kernel, (1, 2, 0))
        kernel_vis = (kernel_vis - kernel_vis.min()) / (kernel_vis.max() - kernel_vis.min())
        
        # 拡大表示
        kernel_large = np.repeat(np.repeat(kernel_vis, 20, axis=0), 20, axis=1)
        axes[1, i+1].imshow(kernel_large)
        axes[1, i+1].set_title(f'Filter #{ch}', fontsize=11)
        axes[1, i+1].axis('off')
    
    plt.suptitle('特徴マップとカーネルの対応関係', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

---

## 5. 学習前 vs 学習後の比較

ランダム初期化されたカーネルと、学習後のカーネルを比較します。

In [ ]:
if TORCH_AVAILABLE:
    # 未学習のVGG16を作成
    vgg16_untrained = models.vgg16(weights=None)
    
    # 第1層カーネルを取得
    untrained_kernels = vgg16_untrained.features[0].weight.data.cpu().numpy()
    trained_kernels = vgg_kernels
    
    # 比較可視化
    fig, axes = plt.subplots(4, 16, figsize=(20, 5))
    
    # 上2行：未学習
    for i in range(32):
        row = i // 16
        col = i % 16
        kernel = untrained_kernels[i]
        kernel_vis = np.transpose(kernel, (1, 2, 0))
        kernel_vis = (kernel_vis - kernel_vis.min()) / (kernel_vis.max() - kernel_vis.min() + 1e-8)
        axes[row, col].imshow(kernel_vis)
        axes[row, col].axis('off')
    
    # 下2行：学習済み
    for i in range(32):
        row = 2 + i // 16
        col = i % 16
        kernel = trained_kernels[i]
        kernel_vis = np.transpose(kernel, (1, 2, 0))
        kernel_vis = (kernel_vis - kernel_vis.min()) / (kernel_vis.max() - kernel_vis.min() + 1e-8)
        axes[row, col].imshow(kernel_vis)
        axes[row, col].axis('off')
    
    axes[0, 0].set_ylabel('未学習', fontsize=12)
    axes[2, 0].set_ylabel('学習済み', fontsize=12)
    
    plt.suptitle('ランダム初期化 vs 学習済みカーネル（VGG16 第1層）', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\n観察:")
    print("- 未学習: ランダムなノイズパターン（構造なし）")
    print("- 学習済み: 明確な構造（エッジ、色のパターン）")
    print("- 学習によって『意味のある』フィルタが創発する")

In [ ]:
if TORCH_AVAILABLE:
    # カーネルの統計的特性の比較
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 重みの分布
    ax1 = axes[0]
    ax1.hist(untrained_kernels.flatten(), bins=50, alpha=0.7, label='未学習', density=True)
    ax1.hist(trained_kernels.flatten(), bins=50, alpha=0.7, label='学習済み', density=True)
    ax1.set_xlabel('重み値')
    ax1.set_ylabel('密度')
    ax1.set_title('重みの分布', fontsize=12)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 各フィルタの標準偏差
    ax2 = axes[1]
    untrained_stds = [untrained_kernels[i].std() for i in range(64)]
    trained_stds = [trained_kernels[i].std() for i in range(64)]
    ax2.scatter(range(64), untrained_stds, alpha=0.7, label='未学習')
    ax2.scatter(range(64), trained_stds, alpha=0.7, label='学習済み')
    ax2.set_xlabel('フィルタ番号')
    ax2.set_ylabel('標準偏差')
    ax2.set_title('各フィルタの重みの標準偏差', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # フィルタ間の相関
    ax3 = axes[2]
    trained_flat = trained_kernels.reshape(64, -1)
    correlation = np.corrcoef(trained_flat)
    im = ax3.imshow(correlation, cmap='RdBu', vmin=-1, vmax=1)
    ax3.set_xlabel('フィルタ番号')
    ax3.set_ylabel('フィルタ番号')
    ax3.set_title('学習済みフィルタ間の相関', fontsize=12)
    plt.colorbar(im, ax=ax3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n統計的特徴:")
    print(f"- 未学習の重みの標準偏差: {np.std(untrained_kernels):.4f}")
    print(f"- 学習済みの重みの標準偏差: {np.std(trained_kernels):.4f}")
    print("- 学習済みフィルタは多様性がある（相関が低い）")

---

## 6. 3DGSへの接続：学習されるガウシアン

CNNのカーネルと3D Gaussian Splattingの類似性を考察します。

In [ ]:
# CNNと3DGSの類似性

comparison = """
╔═══════════════════════════════════════════════════════════════════════════════╗
║              CNNカーネル vs 3Dガウシアン：学習される表現                       ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  【共通点】                                                                    ║
║                                                                               ║
║  1. 局所的な重み付け                                                          ║
║     CNN: 3×3などのカーネルで近傍ピクセルを重み付け                            ║
║     3DGS: ガウシアン関数で3D空間の近傍点を重み付け                            ║
║                                                                               ║
║  2. 学習によるパラメータ最適化                                                ║
║     CNN: バックプロパゲーションでカーネル重みを更新                           ║
║     3DGS: 勾配降下でガウシアンの位置・形状・色を更新                          ║
║                                                                               ║
║  3. 階層的/多スケール表現                                                      ║
║     CNN: 深い層ほど大きな受容野（抽象的特徴）                                 ║
║     3DGS: 異なるサイズのガウシアンで細部〜大域を表現                          ║
║                                                                               ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  【相違点】                                                                    ║
║                                                                               ║
║  • CNN: 離散的なグリッド上で動作、カーネルは全画像で共有                      ║
║  • 3DGS: 連続的な3D空間、各ガウシアンは固有のパラメータ                       ║
║                                                                               ║
║  • CNN: 特徴抽出（認識タスク向け）                                            ║
║  • 3DGS: シーン表現（レンダリング向け）                                       ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝
"""
print(comparison)

In [ ]:
# ガウシアンの「学習」を模擬的に可視化

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# ターゲット画像（単純な形状）
target = np.zeros((64, 64))
target[20:45, 20:45] = 1.0

# ランダムなガウシアンの集合（初期状態）
np.random.seed(42)
n_gaussians = 50

def render_gaussians(positions, scales, intensities, size=64):
    """ガウシアンの集合を画像にレンダリング"""
    img = np.zeros((size, size))
    y, x = np.ogrid[:size, :size]
    
    for pos, scale, intensity in zip(positions, scales, intensities):
        gaussian = np.exp(-((x - pos[0])**2 + (y - pos[1])**2) / (2 * scale**2))
        img += intensity * gaussian
    
    return np.clip(img, 0, 1)

# 初期状態（ランダム）
init_positions = np.random.rand(n_gaussians, 2) * 64
init_scales = np.random.rand(n_gaussians) * 5 + 2
init_intensities = np.random.rand(n_gaussians) * 0.5

# 「学習後」状態（ターゲットに合わせて配置）
learned_positions = []
learned_scales = []
learned_intensities = []

for _ in range(n_gaussians):
    if np.random.rand() < 0.7:  # 70%を四角形内に配置
        pos = [np.random.uniform(20, 45), np.random.uniform(20, 45)]
        intensity = np.random.uniform(0.3, 0.6)
    else:  # 30%を外に（背景）
        pos = [np.random.uniform(0, 64), np.random.uniform(0, 64)]
        intensity = np.random.uniform(-0.1, 0.1)
    learned_positions.append(pos)
    learned_scales.append(np.random.uniform(2, 4))
    learned_intensities.append(intensity)

learned_positions = np.array(learned_positions)
learned_scales = np.array(learned_scales)
learned_intensities = np.array(learned_intensities)

# レンダリング
init_render = render_gaussians(init_positions, init_scales, init_intensities)
learned_render = render_gaussians(learned_positions, learned_scales, learned_intensities)

# 可視化
axes[0].imshow(target, cmap='gray', vmin=0, vmax=1)
axes[0].set_title('ターゲット', fontsize=12)
axes[0].axis('off')

axes[1].imshow(init_render, cmap='gray', vmin=0, vmax=1)
axes[1].scatter(init_positions[:, 0], init_positions[:, 1], c='red', s=10, alpha=0.5)
axes[1].set_title('初期状態\n（ランダムなガウシアン）', fontsize=12)
axes[1].axis('off')

axes[2].imshow(learned_render, cmap='gray', vmin=0, vmax=1)
axes[2].scatter(learned_positions[:, 0], learned_positions[:, 1], c='red', s=10, alpha=0.5)
axes[2].set_title('学習後\n（最適化されたガウシアン）', fontsize=12)
axes[2].axis('off')

axes[3].imshow(np.abs(target - learned_render), cmap='hot')
axes[3].set_title('誤差', fontsize=12)
axes[3].axis('off')

plt.suptitle('ガウシアンの「学習」による表現獲得（3DGSの2D簡易版）', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n共通する本質:")
print("- 初期状態はランダム（意味のない構造）")
print("- 学習によってターゲットを表現できる構造を獲得")
print("- CNNもGSも、データから『表現』を学習する")

---

## まとめ

In [ ]:
summary = """
╔═══════════════════════════════════════════════════════════════════════════════╗
║                    Section A 完了：畳み込みの世界への入口                      ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  【学んだこと】                                                                ║
║                                                                               ║
║  80. 畳み込みの直感的理解                                                     ║
║      → 「周囲を見て自分の値を決める」操作                                     ║
║                                                                               ║
║  81. 畳み込みの数学的定義                                                     ║
║      → 数式、パディング、ストライドの意味                                     ║
║                                                                               ║
║  82-83. NumPyスクラッチ実装                                                   ║
║      → forループ版から高速ベクトル化版へ                                      ║
║                                                                               ║
║  84. 古典的フィルタ                                                           ║
║      → 平滑化、エッジ検出、シャープ化の原理                                   ║
║                                                                               ║
║  85. カーネルの可視化と解釈                                                   ║
║      → 学習済みCNNのカーネル、Gaborフィルタとの類似性                         ║
║                                                                               ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  【次のSection B: 受容野と階層的抽象化】                                       ║
║                                                                               ║
║  86. 受容野入門：「視野」という概念                                           ║
║  87. 層を重ねる：深さが生む抽象化                                             ║
║  88. ダウンサンプリング：効率的に視野を広げる                                 ║
║  89. 受容野と3DGS：空間的影響範囲の対比                                       ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝
"""
print(summary)

---

## 次のステップ

**Section B: 受容野と階層的抽象化** に進みます。

次のノートブック **86. 受容野入門** では：

- 受容野（Receptive Field）の直感的理解
- 1つのピクセルが「世界のどこを見ているか」
- 受容野の計算と可視化ツールの作成

畳み込みの「局所性」がどのように「大域的な理解」につながるかを探求します。